In [1]:
import argparse
import dask
import json
import netCDF4 as nc4
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint
import time
import warnings
import xarray as xr
from dask import delayed
import datetime
import random
from collections import OrderedDict

warnings.filterwarnings('ignore')

In [34]:
metadata_dir = Path('/home/ifenty/git_repos_others/ECCO-GROUP/ECCO-ACCESS/metadata/ECCOv4r4_metadata_json')
model_version = 'ECCO_V4r4'

In [3]:
print('loading summary fix')
with open(metadata_dir / 'ECCOv4r4_variable_metadata.json') as f:
    variable_metadata = json.load(f)
    #pprint(len(summary_fix.keys()))
    #pprint(list(summary_fix.keys()))


loading summary fix


In [51]:
long_filename = model_version +'_variable_metadata_description.txt'
tabsv_filename = model_version +'_variable_metadata_description.tabsv'

lines = []
csv_lines = []

for var in variable_metadata:
    name = var['name']
    long_name = var['long_name']
    units = var['units']
    comments_1 = var['comments_1'] 
    comments_2 = var['comments_2']
    dimension = var['grid_dimension']
                   
    lines.append( f'{name}: {long_name}')
    lines.append('==================================================================================')
    lines.append(f'{comments_1}\n\n{comments_2}')
    lines.append(f'\nUnits: {units}')
    lines.append(f'Dimension: {dimension}')
    lines.append('\n')
    
    csv_lines.append(f'{name}\t {long_name} \t {comments_1} \t {comments_2} \t {units} \t {dimension}')
            

In [50]:
with open(metadata_dir / long_filename, 'w') as the_file:
    for i in lines:
        the_file.write(i + '\n')

the_file.close()

with open(metadata_dir / csv_filename, 'w') as the_file:
    for i in csv_lines:
        the_file.write(i + '\n')

the_file.close()

In [33]:
lines

['ADVr_SLT: Vertical advective flux of salinity',
 '==================================================================================',
 "Vertical advective flux of salinity (SALT) in the +z direction through the top 'w' face of the tracer cell on the native model grid\n\nNote: in the Arakawa-C grid, vertical flux quantities are staggered relative to the tracer cells with indexing such that +ADVr_SLT(i,j,k_l) corresponds to upward +z fluxes through the top 'w' face of the tracer cell at (i,j,k). Salinity defined using CF convention 'Sea water salinity is the salt content of sea water, often on the Practical Salinity Scale of 1978. However, the unqualified term 'salinity' is generic and does not necessarily imply any particular method of calculation. The units of salinity are dimensionless and the units attribute should normally be given as 1e-3 or 0.001 i.e. parts per thousand.' see https://cfconventions.org/Data/cf-standard-names/73/build/cf-standard-name-table.html",
 '\nUnits: 1e-3

In [2]:
def load_valid_minmax(valid_minmax_dir):
    valid_minmax_files = list(valid_minmax_dir.glob('**/valid_minmax*.nc'))

    minmax = dict()
    for mmf in valid_minmax_files:
        tmp = xr.open_dataset(mmf)
        ds_id = tmp.attrs['id'].split('/')[1]

        minmax[ds_id] = dict()
        for dv in tmp.data_vars:
            minmax[ds_id][dv] = dict()
            minmax[ds_id][dv]['valid_min'] = tmp[dv].values[0]
            minmax[ds_id][dv]['valid_max'] = tmp[dv].values[1]
    return minmax

In [193]:
def sort_attrs(attrs):
    od = OrderedDict()

    keys = sorted(list(attrs.keys()),key=str.casefold)

    for k in keys:
        od[k] = attrs[k]

    return od


In [257]:
def apply_fixes(ecco_filename, minmax, comment_fix, summary_fix, qc_prob):

    comment_keys = list(comment_fix.keys())
    summary_fix_keys = list(summary_fix.keys())
    minmax_keys = list(minmax.keys())
    
    print('\nApplying fixes for ', ecco_filename.name)
    try:
        # open a dataset
        with nc4.Dataset(ecco_filename, mode='r+') as tmp_ds:
            
            # get variables in this dataset
            nc_dvs = list(tmp_ds.variables)
                
            # get ID, shortname, title
            ds_id        = tmp_ds.id.split('/')[1]
            ds_shortname = tmp_ds.metadata_link.split('ShortName=')[1]
            ds_title     = tmp_ds.title
            
            metadata_updated = False
            
            # fix summary
            if ds_id in summary_fix.keys():
                print('\n>> fixing summary for ', ds_id)
                if ds_shortname == summary_fix[ds_id]['shortname'] and \
                   ds_title     == summary_fix[ds_id]['title']:
                    
                    print('... title and shortname match')
                    
                    if tmp_ds.summary == summary_fix[ds_id]['summary']:
                        print('... new summary == old summary')
                        print('... not updating summary')
                
                    else:
                        print('... new summary != old summary')
                        print('\n... old summary: ', tmp_ds.summary)
                        tmp_ds.setncattr('summary', summary_fix[ds_id]['summary'])
                        print('\n... new summary: ', tmp_ds.summary)
                        metadata_updated = True
            
                else:
                    print(f'\n+ FAILURE: title or shortname do not match in summary_fix[{ds_id}]: {ecco_filename.name}')
                    print('... from file        ', ds_shortname, ds_title)
                    print('... from summary_fix ', summary_fix[ds_id]['shortname'],\
                          summary_fix[ds_id]['title'])
                    return -1

            else:
                print(f'\n+ FAILURE: {ds_id} not in summary_fix.keys(): {ecco_filename.name}')
                return -1
            
            # fix units on select variables
            print('\n>> fixing units')
            if 'EXFatemp' in nc_dvs:
                v = tmp_ds.variables['EXFatemp']
                
                if v.units != 'degree_K':
                    print (f'... fixing units for EXFatemp')
                    print (f'... old {v.units}')
                    v.setncattr("units", 'degree_K')
                    print (f'... new: {v.units}')
                    metadata_updated = True

                else:
                    print (f'... units are identical {v.units} -- not fixing units!')
            else:
                print (f'... EXFatemp not in granule -- not fixing units!')

                
            # fix comment on select variables
            print('\n>> fixing comment')
            vars_to_fix = set(nc_dvs).intersection(set(comment_keys))
            if len(vars_to_fix) > 0:
                for dv in vars_to_fix:
                    v = tmp_ds.variables[dv]
                    print(f'... fixing comment for {dv}')
                    v.setncattr("comment", comment_fix[dv]["comment"])
                    print(f'... new {v.comment}')
                    metadata_updated = True
            else:
                print(f'... granule has no variables with updated comments -- not updating')
                          
                        
            # fix minmax
            print('\n>>fixing minmax:')
            if ds_id in minmax.keys():
                # get variables in the minmax dataset
                minmax_dvs = list(minmax[ds_id].keys())
                
                print('minmax keys: ', minmax_dvs)

                # loop through all variables in the minmax dictionary
                for minmax_dv in minmax_dvs:
                    if minmax_dv in nc_dvs:
                        print(f'\n{minmax_dv} found in nc_dvs')
                        # get a pointer to the this data variable
                        v = tmp_ds.variables[minmax_dv]
                        
                        # pull out the current valid min max attributes
                        old_valid_min = v.valid_min
                        old_valid_max = v.valid_max
                          
                        # get the valid min and max for this variable
                        new_valid_min = minmax[ds_id][minmax_dv]['valid_min']
                        new_valid_max = minmax[ds_id][minmax_dv]['valid_max']
                            
                        # QC section
                        # roll the dice
                        qc_rand = random.random()
                        
                        if qc_rand < qc_prob:
                            # load actual vmin vmax here (expensive IO)
                            v_min = np.nanmin(v[:])
                            v_max = np.nanmax(v[:])

                            print(f'   QC act min/max        : {v_min:.12} {v_max:.12}')
                            print(f'   QC old valid_min/max  : {old_valid_min:.12} {old_valid_max:.12}')
                            print(f'   QC new valid_min/max  : {new_valid_min:.12} {new_valid_max:.12}')

                            if (old_valid_min > v_min) or (old_valid_max < v_max):
                                print(f'   QC old valid min/max was wrong')
                                print(f'     1. old valid min >= vmin {old_valid_min:.12} {v_min:.12}\
                                    {old_valid_min > v_min}')
                                print(f'     2. old valid max <= vmax {old_valid_max:.12} {v_max:.12}\
                                    {old_valid_max < v_max}')
                            else: 
                                print(f'   QC ... old valid min/max was ok')
       
                            if (new_valid_min > v_min) or (new_valid_max < v_max):
                                print(f'\n+ FAILURE: new valid min/max is wrong in {ds_id} {minmax_dv}')
                                print(f'   1. new valid min >= vmin {new_valid_min:.12} {v_min:.12} \
                                    {new_valid_min > v_min}')
                                print(f'   2. new valid max <= vmax {new_valid_max:.12} {v_max:.12} \
                                    {new_valid_max < v_max}')
                                return -1
                            
                            else:  
                                print('   QC ... new valid min/max is ok')
                            print('\n')
                           
                        if (new_valid_min == old_valid_min) and (new_valid_max == old_valid_max):
                            print('... new and old valid min/max are identical!')
                            print('... not updating valid min/max!')

                        else:  
                            v.setncattr("valid_min", new_valid_min)
                            v.setncattr("valid_max", new_valid_max)

                            print(f'... new and old valid min/max are different!')
                            print(f'... old/new valid min {old_valid_min:.12} {v.valid_min:.12}')
                            print(f'... old/new valid max {old_valid_max:.12} {v.valid_max:.12}')

                            metadata_updated = True

                    else:  
                        print(f'\n+ FAILURE: minmax key {minmax_dv} not in granule variables {nc_dvs}: {ecco_filename.name}')
                        return -1
                
            else:  
                print(f'\n!!!! granule id not found in minmax keys {ds_id}')
                return -1
                             
            
            # update the reference to the URS approved version
            print ('\n>> fixing references')
            tmp_ds.setncattr('references', \
                             'ECCO Consortium, Fukumori, I., Wang, O., Fenty, I., Forget, G., Heimbach, P., & Ponte, R. M. 2020. Synopsis of the ECCO Central Production Global Ocean and Sea-Ice State Estimate (Version 4 Release 4). doi:10.5281/zenodo.3765928')
            tmp_ds.setncattr('source', \
                             'The ECCO V4r4 state estimate was produced by fitting a free-running solution of the MITgcm (checkpoint 66g) to satellite and in situ observational data in a least squares sense using the adjoint method')
            metadata_updated = True

            
            if metadata_updated:
                # update date of modified metadata
                current_time = datetime.datetime.now().isoformat()[0:19]
                tmp_ds.setncattr('date_modified', current_time)
                tmp_ds.setncattr('date_metadata_modified', current_time)

                # alphabetically sort all attributes
                sorted_attr_dict = sort_attrs(tmp_ds.__dict__)

                # delete all attributes one at a time
                for attr in tmp_ds.ncattrs():
                    tmp_ds.delncattr(attr)
                
                # replace all one at a time (in alphabetical order)
                for attr in sorted_attr_dict:
                    tmp_ds.setncattr(attr, sorted_attr_dict[attr])

                print(f"\n+ SUCCESS: changes applied {ecco_filename.name}\n")
                return 1
            else:
                print(f"\n+ SUCCESS: no changes applied to {ecco_filename.name}\n")
                return 0

    except Exception as e:
        raise e
    
    print('could not open file!')
    return -1 

In [39]:
def f1(ecco_files, minmax, comment_fix, summary_fix, qc_prob):
    results = []
    for ecco_filename in ecco_files:
        result = apply_fixes(ecco_filename, minmax, comment_fix, summary_fix, qc_prob)
        results.append(result)
    return results

In [40]:
def get_groupings(base_dir, grid_type, time_type):
    groupings = dict()
    tmp = Path(f'{base_dir}/{grid_type}/{time_type}')
    print(tmp)
    if tmp.exists():
        g_dirs = np.sort(list(tmp.iterdir()))
        for pi, p in enumerate(g_dirs):
            grouping = str(p).split('/')[-1]
            groupings[pi] = dict()
            groupings[pi]['name'] = grouping
            groupings[pi]['grid'] = grid_type
            groupings[pi]['time_type'] = time_type
            groupings[pi]['directory'] = p
            
    return groupings

## Inputs

In [268]:
grids = ['native','latlon']
time_avgs = ['day_inst', 'day_mean','mon_mean']

In [269]:
dataset_base_dir = Path('/home/ifenty/ian1/ifenty/ECCOv4/Version4/Release4/podaac_test/podaac/')

In [273]:
time_type = time_avgs[2]
grid_type = grids[1]
groupings = get_groupings(dataset_base_dir, grid_type, time_type)

/home/ifenty/ian1/ifenty/ECCOv4/Version4/Release4/podaac_test/podaac/latlon/mon_mean


In [274]:
for gi in groupings:
    print(gi, groupings[gi]['name'])

0 ATM_SURFACE_TEMP_HUM_WIND_PRES
1 OCEAN_AND_ICE_SURFACE_FW_FLUX
2 OCEAN_AND_ICE_SURFACE_HEAT_FLUX
3 OCEAN_AND_ICE_SURFACE_STRESS
4 OCEAN_BOLUS_VELOCITY
5 OCEAN_BOTTOM_PRESSURE
6 OCEAN_DENS_STRAT_PRESS
7 OCEAN_MIXED_LAYER_DEPTH
8 OCEAN_TEMPERATURE_SALINITY
9 OCEAN_VELOCITY
10 SEA_ICE_CONC_THICKNESS
11 SEA_ICE_VELOCITY
12 SEA_SURFACE_HEIGHT


In [291]:
gi = 8
print(groupings[gi])
grouping_info = groupings[gi]

{'name': 'OCEAN_TEMPERATURE_SALINITY', 'grid': 'latlon', 'time_type': 'mon_mean', 'directory': PosixPath('/home/ifenty/ian1/ifenty/ECCOv4/Version4/Release4/podaac_test/podaac/latlon/mon_mean/OCEAN_TEMPERATURE_SALINITY')}


In [292]:
summary_fix_dir = Path('/home/ifenty/git_repos_others/ECCO-GROUP/ECCO-ACCESS/metadata/ECCOv4r4_metadata_json')
with open(summary_fix_dir / 'ECCOv4r4_dataset_summary.json') as f:
  summary_fix = json.load(f)
pprint(len(summary_fix.keys()))
pprint(list(summary_fix.keys()))

79
['ECG5D-ATM44',
 'ECG5M-ATM44',
 'ECL5D-ATM44',
 'ECL5M-ATM44',
 'ECG5D-BOL44',
 'ECG5M-BOL44',
 'ECL5D-BOL44',
 'ECL5M-BOL44',
 'ECG5D-ODE44',
 'ECG5M-ODE44',
 'ECL5D-ODE44',
 'ECL5M-ODE44',
 'ECG5D-FRE44',
 'ECG5M-FRE44',
 'ECL5D-FRE44',
 'ECL5M-FRE44',
 'ECTSS-MAP44',
 'ECTSD-MSL44',
 'ECTSM-MSL44',
 'ECG5D-HEA44',
 'ECG5M-HEA44',
 'ECL5D-HEA44',
 'ECL5M-HEA44',
 'ECG5D-OML44',
 'ECG5M-OML44',
 'ECL5D-OML44',
 'ECL5M-OML44',
 'ECG5D-OBP44',
 'ECG5M-OBP44',
 'ECL5D-OBP44',
 'ECL5M-OBP44',
 'ECL5S-OBP44',
 'ECL5D-3MT44',
 'ECL5M-3MT44',
 'ECL5D-3SF44',
 'ECL5M-3SF44',
 'ECL5D-3TF44',
 'ECL5M-3TF44',
 'ECL5D-3VF44',
 'ECL5M-3VF44',
 'ECL5D-STF44',
 'ECL5M-STF44',
 'ECG5D-OVE44',
 'ECG5M-OVE44',
 'ECL5D-OVE44',
 'ECL5M-OVE44',
 'ECTSS-SBO44',
 'ECG5D-ICO44',
 'ECG5M-ICO44',
 'ECL5D-ICO44',
 'ECL5M-ICO44',
 'ECL5S-ICO44',
 'ECL5D-SIH44',
 'ECL5M-SIH44',
 'ECL5D-ISP44',
 'ECL5M-ISP44',
 'ECG5D-SIV44',
 'ECG5M-SIV44',
 'ECL5D-SIV44',
 'ECL5M-SIV44',
 'ECL5S-SIV44',
 'ECG5D-SSH44',
 'ECG

In [293]:
valid_minmax_dir = Path('/home/ifenty/ian1/ifenty/ECCOv4/Version4/Release4/valid_minmax/valid_minmax_final')
minmax = load_valid_minmax(valid_minmax_dir)
print(len(minmax.keys()))
pprint(list(minmax.keys()))

71
['ECL5S-SSH44',
 'ECL5S-ICO44',
 'ECL5S-OTS44',
 'ECL5S-OBP44',
 'ECL5S-SIV44',
 'ECL5D-SIH44',
 'ECL5D-OBP44',
 'ECL5D-ODE44',
 'ECL5D-STF44',
 'ECL5D-OML44',
 'ECL5D-HEA44',
 'ECL5D-ICO44',
 'ECL5D-STR44',
 'ECL5D-3MT44',
 'ECL5D-ATM44',
 'ECL5D-BOL44',
 'ECL5D-OVE44',
 'ECL5D-FRE44',
 'ECL5D-SSH44',
 'ECL5D-3VF44',
 'ECL5D-SIV44',
 'ECL5D-OTS44',
 'ECL5D-3TF44',
 'ECL5D-3SF44',
 'ECL5D-ISP44',
 'ECL5M-OVE44',
 'ECL5M-SIH44',
 'ECL5M-ATM44',
 'ECL5M-BOL44',
 'ECL5M-SIV44',
 'ECL5M-ICO44',
 'ECL5M-ISP44',
 'ECL5M-3VF44',
 'ECL5M-3SF44',
 'ECL5M-HEA44',
 'ECL5M-OBP44',
 'ECL5M-3TF44',
 'ECL5M-SSH44',
 'ECL5M-STR44',
 'ECL5M-FRE44',
 'ECL5M-OTS44',
 'ECL5M-ODE44',
 'ECL5M-3MT44',
 'ECL5M-OML44',
 'ECL5M-STF44',
 'ECG5D-HEA44',
 'ECG5D-SIV44',
 'ECG5D-STR44',
 'ECG5D-OBP44',
 'ECG5D-OTS44',
 'ECG5D-ODE44',
 'ECG5D-FRE44',
 'ECG5D-OML44',
 'ECG5D-SSH44',
 'ECG5D-OVE44',
 'ECG5D-ATM44',
 'ECG5D-ICO44',
 'ECG5D-BOL44',
 'ECG5M-SIV44',
 'ECG5M-ICO44',
 'ECG5M-OVE44',
 'ECG5M-OTS44',
 'ECG

In [294]:
set_a = set(list(summary_fix.keys()))
set_b= set(list(minmax.keys()))

In [295]:
sd = set_b.symmetric_difference(set_a)
for s in sd:
    print(summary_fix[s]['title'])

ECCO Global Mean Atmospheric Pressure - Snapshot (Version 4 Release 4)
ECCO Global Mean Sea Level - Monthly Mean (Version 4 Release 4)
ECCO SBO Core Products - Snapshot (Version 4 Release 4)
ECCO Geometry Parameters for the 0.5 degree Lat-Lon Model Grid (Version 4 Release 4)
ECCO Geometry Parameters for the Lat-Lon-Cap 90 (llc90) Native Model Grid (Version 4 Release 4)
ECCO Ocean 3D Gent-Mcwilliams, Redi, and Background Vertical Diffusivity Coefficients for the 0.5 degree Lat-Lon Model Grid (Version 4 Release 4)
ECCO Ocean 3D Gent-Mcwilliams, Redi, and Background Vertical Diffusivity Coefficients for the Lat-Lon-Cap 90 (llc90) Native Model Grid (Version 4 Release 4)
ECCO Global Mean Sea Level - Daily Mean (Version 4 Release 4)


## Calc

In [296]:
glob_name = '**/*ECCO_V4r4*nc'
ecco_files = np.sort(list(grouping_info['directory'].glob(glob_name)))
print(len(ecco_files))
ecco_files[0]

12


PosixPath('/home/ifenty/ian1/ifenty/ECCOv4/Version4/Release4/podaac_test/podaac/latlon/mon_mean/OCEAN_TEMPERATURE_SALINITY/OCEAN_TEMPERATURE_SALINITY_mon_mean_1992-01_ECCO_V4r4_latlon_0p50deg.nc')

In [297]:
qc_prob = 1

In [298]:
start_time = time.time()
f1_out = f1(ecco_files, minmax, comment_fix, summary_fix, qc_prob)
delta_time = time.time() - start_time
time_per = delta_time / len(ecco_files)


Applying fixes for  OCEAN_TEMPERATURE_SALINITY_mon_mean_1992-01_ECCO_V4r4_latlon_0p50deg.nc

>> fixing summary for  ECG5M-OTS44
... title and shortname match
... new summary == old summary
... not updating summary

>> fixing units
... EXFatemp not in granule -- not fixing units!

>> fixing comment
... fixing comment for SALT
... new Defined using CF convention 'Sea water salinity is the salt content of sea water, often on the Practical Salinity Scale of 1978. However, the unqualified term 'salinity' is generic and does not necessarily imply any particular method of calculation. The units of salinity are dimensionless and the units attribute should normally be given as 1e-3 or 0.001 i.e. parts per thousand.' see https://cfconventions.org/Data/cf-standard-names/73/build/cf-standard-name-table.html

>>fixing minmax:
minmax keys:  ['THETA', 'SALT']

THETA found in nc_dvs
   QC act min/max        : -1.9999768734 31.5259037018
   QC old valid_min/max  : -2.29093885422 36.0329551697
   QC ne

   QC act min/max        : -2.17186188698 32.3803939819
   QC old valid_min/max  : -2.29093885422 36.0329551697
   QC new valid_min/max  : -2.29093885422 36.0329551697
   QC ... old valid min/max was ok
   QC ... new valid min/max is ok


... new and old valid min/max are identical!
... not updating valid min/max!

SALT found in nc_dvs
   QC act min/max        : 22.5017795563 40.6417999268
   QC old valid_min/max  : 17.1066379547 41.2680244446
   QC new valid_min/max  : 17.1066379547 41.2680244446
   QC ... old valid min/max was ok
   QC ... new valid min/max is ok


... new and old valid min/max are identical!
... not updating valid min/max!

>> fixing references

+ SUCCESS: changes applied OCEAN_TEMPERATURE_SALINITY_mon_mean_1992-06_ECCO_V4r4_latlon_0p50deg.nc


Applying fixes for  OCEAN_TEMPERATURE_SALINITY_mon_mean_1992-07_ECCO_V4r4_latlon_0p50deg.nc

>> fixing summary for  ECG5M-OTS44
... title and shortname match
... new summary == old summary
... not updating summary

>> fixing 

   QC act min/max        : -1.9651184082 31.0659751892
   QC old valid_min/max  : -2.29093885422 36.0329551697
   QC new valid_min/max  : -2.29093885422 36.0329551697
   QC ... old valid min/max was ok
   QC ... new valid min/max is ok


... new and old valid min/max are identical!
... not updating valid min/max!

SALT found in nc_dvs
   QC act min/max        : 22.3869552612 40.6413459778
   QC old valid_min/max  : 17.1066379547 41.2680244446
   QC new valid_min/max  : 17.1066379547 41.2680244446
   QC ... old valid min/max was ok
   QC ... new valid min/max is ok


... new and old valid min/max are identical!
... not updating valid min/max!

>> fixing references

+ SUCCESS: changes applied OCEAN_TEMPERATURE_SALINITY_mon_mean_1992-11_ECCO_V4r4_latlon_0p50deg.nc


Applying fixes for  OCEAN_TEMPERATURE_SALINITY_mon_mean_1992-12_ECCO_V4r4_latlon_0p50deg.nc

>> fixing summary for  ECG5M-OTS44
... title and shortname match
... new summary == old summary
... not updating summary

>> fixing u

In [303]:
str(f1_out)

'[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]'

In [300]:
print(delta_time)
print(time_per)
print(time_per*9000)

17.376691818237305
1.4480576515197754
13032.518863677979


In [301]:
ecco_field = xr.open_dataset(ecco_files[0])
ecco_field.close()

In [302]:
ecco_field.attrs

{'acknowledgement': 'This research was carried out by the Jet Propulsion Laboratory, managed by the California Institute of Technology under a contract with the National Aeronautics and Space Administration.',
 'author': 'Ian Fenty and Ou Wang',
 'cdm_data_type': 'Grid',
 'comment': 'Fields provided on a regular lat-lon grid. They have been mapped to the regular lat-lon grid from the original ECCO lat-lon-cap 90 (llc90) native model grid.',
 'Conventions': 'CF-1.8, ACDD-1.3',
 'coordinates_comment': "Note: the global 'coordinates' attribute describes auxillary coordinates.",
 'creator_email': 'ecco-group@mit.edu',
 'creator_institution': 'NASA Jet Propulsion Laboratory (JPL)',
 'creator_name': 'ECCO Consortium',
 'creator_type': 'group',
 'creator_url': 'https://ecco-group.org',
 'date_created': '2020-12-18T09:50:18',
 'date_issued': '2020-12-18T09:50:18',
 'date_metadata_modified': '2021-03-14T17:02:20',
 'date_modified': '2021-03-14T17:02:20',
 'geospatial_bounds_crs': 'EPSG:4326',
 